In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time


#open the webpage and remove cookies and ads    

url = "https://www.premierleague.com/tables"
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
accept_cookies =  driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
accept_cookies.click()
#remove = driver.find_element(By.XPATH, '//*[@id="advertClose"]')
#remove.click()

#years = ['2020/21','2021/22']
years = ['2020/21']
i=2019

for season in years:
    i=i+1
    print(season)

    
    file1 = open(r"C:\Users\matts\Documents\Empirical Project\EmpiricalProject-720005462\LeagueTable" + str(i) + ".csv", 'w')
    file1.write('Position, Club Name, Points, Goals For\n')
   

    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@data-dropdown-current='compSeasons']  ")))
    element.click()
    year_to_select = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.XPATH, f"//li[text()='{season}']"))
    )
    year_to_select.click()
    #Wait for page to refresh
   
    time.sleep(10)

    #Get the data
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
   
    # Find the body of the league table
    table_body = soup.find('tbody', class_='league-table__tbody')

    # Iterate through each row in the table body
    for row in table_body.find_all('tr'):
       if row.find('td', class_='league-table__pos'):
        #position = row.find('td', class_='league-table__pos').text.strip()
        position = row.find('span', class_='league-table__value value').text.strip()    
        club_name = row.find('span', class_='league-table__team-name--long').text.strip()
        #played = row.find_all('td')[2].text.strip()
        #won = row.find_all('td')[3].text.strip()
        #drawn = row.find_all('td')[4].text.strip()
        #lost = row.find_all('td')[5].text.strip()
        gf = row.find_all('td')[6].text.strip()
        #ga = row.find_all('td')[7].text.strip()
        #gd = row.find_all('td')[8].text.strip()
        points = row.find('td', class_='league-table__points').text.strip() if row.find('td', class_='league-table__points') else 'N/A'
        #print(f'{position} {club_name} - Played: {played}, Wins: {won}, Draws: {drawn}, Losses: {lost}, GF: {gf}, GA: {ga}, GD: {gd}, Points: {points}')
        print(f'Position: {position}, Club: {club_name}, Points: {points}')
        file1.write(f'{position}, {club_name}, {points}, {gf}\n')

    file1.close()

driver.quit()

2020/21
Position: 1, Club: Manchester City, Points: 86
Position: 2, Club: Manchester United, Points: 74
Position: 3, Club: Liverpool, Points: 69
Position: 4, Club: Chelsea, Points: 67
Position: 5, Club: Leicester City, Points: 66
Position: 6, Club: West Ham United, Points: 65
Position: 7, Club: Tottenham Hotspur, Points: 62
Position: 8, Club: Arsenal, Points: 61
Position: 9, Club: Leeds United, Points: 59
Position: 10, Club: Everton, Points: 59
Position: 11, Club: Aston Villa, Points: 55
Position: 12, Club: Newcastle United, Points: 45
Position: 13, Club: Wolverhampton Wanderers, Points: 45
Position: 14, Club: Crystal Palace, Points: 44
Position: 15, Club: Southampton, Points: 43
Position: 16, Club: Brighton and Hove Albion, Points: 41
Position: 17, Club: Burnley, Points: 39
Position: 18, Club: Fulham, Points: 28
Position: 19, Club: West Bromwich Albion, Points: 26
Position: 20, Club: Sheffield United, Points: 23


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time

def scrape_premier_league_players(soup,player_file):
   
    #Extract and print information for each player

    player_rows = soup.find_all('tr', class_='player')

    for row in player_rows:
        player_name_link = row.find('a', class_='player__name')
        if player_name_link:
            player_name = player_name_link.get_text(strip=True)
            player_href = player_name_link.get('href')
       
        position = row.find('td', class_='player__position').get_text(strip=True)
        country = row.find('span', class_='player__country').get_text(strip=True)
        
        
        print(f'Name: {player_name}, Position: {position}, Country: {country}')
        player_file.write(f"Name: {player_name}, Position: {position}, Country: {country},")
        
        
        scrape_premier_league_players_stats_2(player_href,player_file)


        

def scrape_premier_league_players_stats_2(player_page,player_file):

    player_page="https:"+player_page

    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(player_page)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
 
          
    labels = soup.find_all('div', class_='player-overview__label')
    clubs_rows = soup.find_all('tr', class_='table player-club-history__table-row')
                            

    player_stats = {}
    club_name=""
    height=""
    DoB=""


    for row in clubs_rows:
        season = row.find('td', class_='player-club-history__season')
        if season and season.get_text(strip=True) == '2020/2021':
            club_name = row.find('span', class_='player-club-history__team-name').get_text(strip=True)
            break
           
    #print("Club Name:", club_name)


    # Loop through all found labels to find "Height"
    for label in labels:
        if label.get_text(strip=True).lower() == "height":
            # Find the next relate4 div that has class 'player-overview__info'
            info_div = label.find_next_sibling('div', class_='player-overview__info')
            if info_div:
                height = info_div.get_text(strip=True)
                break

    # Print the player height name
    #print("Height:", height)


    # Loop through all found labels to find DoB
    for label in labels:
        if label.get_text(strip=True).lower() == "date of birth":
            # Find the next relate4 div that has class 'player-overview__info'
            info_div = label.find_next_sibling('div', class_='player-overview__info')
            if info_div:
                DoB = info_div.get_text(strip=True)
                break

    # Print the player height name
    #print("DoB:", DoB)


    #player_file.write(f",Club: {club_name}, Height: {height}, DoB: {DoB}\n")
    #player_file.write(f",Club: {club_name}, Height: {height}, DoB: {DoB}")

    #print(f'Name: {player_name}, Position: {position}, Country: {country}, Club: {club_name}, Height: {height}, DoB: {DoB}')
    #player_file.write(f"Name: {player_name}, Position: {position}, Country: {country}, Club: {club_name}, Height: {height}, DoB: {DoB}")

    print(f'Club: {club_name}, Height: {height}, DoB: {DoB}')
    player_file.write(f"Club: {club_name}, Height: {height}, DoB: {DoB}\n")

   
    driver.quit()




#open the webpage and remove cookies and ads    

url = "https://www.premierleague.com/players"
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
accept_cookies =  driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
accept_cookies.click()
#remove = driver.find_element(By.XPATH, '//*[@id="advertClose"]')
#remove.click()    

#years = ['1992/93','1993/94']
#i=1991

#years = ['2020/21','2021/22']
years = ['2020/21']
i=2019

for season in years:
    i=i+1
    print(season)

 #Open a file

    file1 = open(r"C:\Users\matts\Documents\Empirical Project\EmpiricalProject-720005462\PlayerData" + str(i) + ".csv", 'w')

   

#Select the year to process

    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@data-dropdown-current='compSeasons']  ")))
    element.click()
    year_to_select = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.XPATH, f"//li[text()='{season}']"))
    )
    year_to_select.click()



#Scroll to bottom and wait 60 seconds
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#time.sleep(60)
   
#Wait for page to refresh
   
    time.sleep(10)



#Get the data
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    scrape_premier_league_players(soup,file1)
    print()

#Close the file
    file1.close()

#  Close the browser when done

driver.quit()

2020/21
Name: Tammy Abraham, Position: Forward, Country: England
Club: Chelsea, Height: 190cm, DoB: 02/10/1997
Name: Ché Adams, Position: Forward, Country: Scotland
Club: Southampton, Height: 175cm, DoB: 13/07/1996
Name: Tosin Adarabioyo, Position: Defender, Country: England
Club: Fulham, Height: 196cm, DoB: 24/09/1997  (26)
Name: Dennis Adeniran, Position: Midfielder, Country: England
Club: Everton, Height: 180cm, DoB: 02/01/1999
Name: Adrián, Position: Goalkeeper, Country: Spain
Club: Liverpool, Height: 190cm, DoB: 03/01/1987  (37)
Name: Adrien Silva, Position: Midfielder, Country: Portugal
Club: Leicester City, Height: 171cm, DoB: 15/03/1989
Name: Oladapo Afolayan, Position: Forward, Country: England
Club: West Ham United, Height: 180cm, DoB: 11/09/1997
Name: Sergio Agüero, Position: Forward, Country: Argentina
Club: Manchester City, Height: 173cm, DoB: 02/06/1988
Name: Ahmed El Mohamady, Position: Defender, Country: Egypt
Club: Aston Villa, Height: 183cm, DoB: 09/09/1987
Name: Ahme

In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time

def scrape_premier_league_players(soup,player_file):
   
    #Extract and print information for each player

    player_rows = soup.find_all('tr', class_='player')

    for row in player_rows:
        player_name_link = row.find('a', class_='player__name')
        if player_name_link:
            player_name = player_name_link.get_text(strip=True)
            player_href = player_name_link.get('href')
       
        position = row.find('td', class_='player__position').get_text(strip=True)
        country = row.find('span', class_='player__country').get_text(strip=True)
        
        
        print(f'Name: {player_name}, Position: {position}, Country: {country}')
        player_file.write(f"{player_name}, {position}, {country},")
        
        
        scrape_premier_league_players_stats_2(player_href,player_file)


        

def scrape_premier_league_players_stats_2(player_page,player_file):

    player_page="https:"+player_page

    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(player_page)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
 
          
    labels = soup.find_all('div', class_='player-overview__label')
    clubs_rows = soup.find_all('tr', class_='table player-club-history__table-row')
                            

    club_name=""
    height=""
    DoB=""
    goals=""


    for row in clubs_rows:
        season = row.find('td', class_='player-club-history__season')
        if season and season.get_text(strip=True) == '2020/2021':
            club_name = row.find('span', class_='player-club-history__team-name').get_text(strip=True)
            goals = row.find('td', class_='player-club-history__goals').get_text(strip=True)
            break
           


    # Loop through all found labels to find "Height"
    for label in labels:
        if label.get_text(strip=True).lower() == "height":
            # Find the next relate4 div that has class 'player-overview__info'
            info_div = label.find_next_sibling('div', class_='player-overview__info')
            if info_div:
                height = info_div.get_text(strip=True)
                break


    
    # Loop through all found labels to find DoB
    for label in labels:
        if label.get_text(strip=True).lower() == "date of birth":
            # Find the next relate4 div that has class 'player-overview__info'
            info_div = label.find_next_sibling('div', class_='player-overview__info')
            if info_div:
                DoB = info_div.get_text(strip=True)
                break


    print(f'Club: {club_name}, Height: {height}, DoB: {DoB}, Goals: {goals}')
    player_file.write(f"{club_name}, {height}, {DoB}, {goals}\n")

   
    driver.quit()




#open the webpage and remove cookies and ads    

url = "https://www.premierleague.com/players"
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
accept_cookies =  driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
accept_cookies.click()
#remove = driver.find_element(By.XPATH, '//*[@id="advertClose"]')
#remove.click()    

#years = ['1992/93','1993/94']
#i=1991

#years = ['2020/21','2021/22']
years = ['2020/21']
i=2019

for season in years:
    i=i+1
    print(season)

 #Open a file

    file1 = open(r"C:\Users\matts\Documents\Empirical Project\EmpiricalProject-720005462\PlayerData" + str(i) + ".csv", 'w')
    file1.write('Player Name, Position, Nationality, Club, Height, DoB, Goals\n')

   

#Select the year to process

    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@data-dropdown-current='compSeasons']  ")))
    element.click()
    year_to_select = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.XPATH, f"//li[text()='{season}']"))
    )
    year_to_select.click()



#Scroll to bottom and wait 60 seconds
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#time.sleep(60)
   
#Wait for page to refresh
   
    time.sleep(10)



#Get the data
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    scrape_premier_league_players(soup,file1)
    print()

#Close the file
    file1.close()

#  Close the browser when done

driver.quit()

2020/21
Name: Tammy Abraham, Position: Forward, Country: England
Club: Chelsea, Height: 190cm, DoB: 02/10/1997, Goals: 6
Name: Ché Adams, Position: Forward, Country: Scotland
Club: Southampton, Height: 175cm, DoB: 13/07/1996, Goals: 9
Name: Tosin Adarabioyo, Position: Defender, Country: England
Club: Fulham, Height: 196cm, DoB: 24/09/1997  (26), Goals: 0
Name: Dennis Adeniran, Position: Midfielder, Country: England
Club: Everton, Height: 180cm, DoB: 02/01/1999, Goals: 0
Name: Adrián, Position: Goalkeeper, Country: Spain
Club: Liverpool, Height: 190cm, DoB: 03/01/1987  (37), Goals: 0
Name: Adrien Silva, Position: Midfielder, Country: Portugal
Club: Leicester City, Height: 171cm, DoB: 15/03/1989, Goals: 0
Name: Oladapo Afolayan, Position: Forward, Country: England
Club: West Ham United, Height: 180cm, DoB: 11/09/1997, Goals: 0
Name: Sergio Agüero, Position: Forward, Country: Argentina
Club: Manchester City, Height: 173cm, DoB: 02/06/1988, Goals: 4
Name: Ahmed El Mohamady, Position: Defen

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time


#open the webpage and remove cookies and ads    

url = "https://www.premierleague.com/tables"
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
accept_cookies =  driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
accept_cookies.click()
#remove = driver.find_element(By.XPATH, '//*[@id="advertClose"]')
#remove.click()

#years = ['2020/21','2021/22']
years = ['2020/21']
i=2019

for season in years:
    i=i+1
    print(season)

    
    file1 = open(r"C:\Users\matts\Documents\Empirical Project\EmpiricalProject-720005462\LeagueTable" + str(i) + ".csv", 'w')
    file1.write('Position, Club Name, Points, Goals For\n')
   

    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@data-dropdown-current='compSeasons']  ")))
    element.click()
    year_to_select = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.XPATH, f"//li[text()='{season}']"))
    )
    year_to_select.click()
    #Wait for page to refresh
   
    time.sleep(10)

    #Get the data
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
   
    # Find the body of the league table
    table_body = soup.find('tbody', class_='league-table__tbody')

    # Iterate through each row in the table body
    for row in table_body.find_all('tr'):
       if row.find('td', class_='league-table__pos'):
        #position = row.find('td', class_='league-table__pos').text.strip()
        position = row.find('span', class_='league-table__value value').text.strip()    
        club_name = row.find('span', class_='league-table__team-name--long').text.strip()
        #played = row.find_all('td')[2].text.strip()
        #won = row.find_all('td')[3].text.strip()
        #drawn = row.find_all('td')[4].text.strip()
        #lost = row.find_all('td')[5].text.strip()
        gf = row.find_all('td')[6].text.strip()
        #ga = row.find_all('td')[7].text.strip()
        #gd = row.find_all('td')[8].text.strip()
        points = row.find('td', class_='league-table__points').text.strip() if row.find('td', class_='league-table__points') else 'N/A'
        #print(f'{position} {club_name} - Played: {played}, Wins: {won}, Draws: {drawn}, Losses: {lost}, GF: {gf}, GA: {ga}, GD: {gd}, Points: {points}')
        print(f'Position: {position}, Club: {club_name}, Points: {points}')
        file1.write(f'{position}, {club_name}, {points}, {gf}\n')

    file1.close()

driver.quit()

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time


#open the webpage and accept cookies 

url = "https://www.premierleague.com/tables"
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
accept_cookies =  driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
accept_cookies.click()
remove = driver.find_element(By.XPATH, '//*[@id="advertClose"]')
remove.click()  


#define the start and end years for full years of data and create empty list

start_year = 1992
end_year = 2022
years = []

#loop through the range of years and generate list of all years in correct format for filter
for year in range(start_year, end_year + 1):
    formatted_year = f"{year}/{(year + 1) % 100:02d}"
    years.append(formatted_year)

#variable for file naming
i=1991

#loop through all the years
for season in years:
    i=i+1

    #create csv file with consistent naming and clear directory, to write scraped data into and create 'headers' 
    file1 = open(r"C:\Users\matts\Documents\Empirical Project\EmpiricalProject-720005462\LeagueData\LeagueTable" + str(i) + ".csv", 'w')
    file1.write('Position,Club Name,Points,Goals For\n')
   

    #apply filter to access desired data
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@data-dropdown-current='compSeasons']  ")))
    element.click()
    year_to_select = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.XPATH, f"//li[text()='{season}']"))
    )
    year_to_select.click()
    
    
    #wait for page to refresh
    time.sleep(10)

    #get the data
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
   
    #find the body of the league table
    table_body = soup.find('tbody', class_='league-table__tbody')

    #iterate through each row in the table body
    for row in table_body.find_all('tr'):
       if row.find('td', class_='league-table__pos'):
        #position = row.find('td', class_='league-table__pos').text.strip()
        position = row.find('span', class_='league-table__value value').text.strip()    
        club_name = row.find('span', class_='league-table__team-name--long').text.strip()
        gf = row.find_all('td')[6].text.strip()
        points = row.find('td', class_='league-table__points').text.strip() if row.find('td', class_='league-table__points') else 'N/A'
        
        #write generated data into csv
        file1.write(f'{position},{club_name},{points},{gf}\n')

    #close file and repeat loop
    file1.close()

#close webpage
driver.quit()

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time


#dfefine function
def scrape_premier_league_players(soup,player_file):
   
    #extract and print information for each player
    player_rows = soup.find_all('tr', class_='player')

    #scrape player name and access href to gain access to further stats
    for row in player_rows:
        player_name_link = row.find('a', class_='player__name')
        if player_name_link:
            player_name = player_name_link.get_text(strip=True)
            player_href = player_name_link.get('href')

        #scrape position and nationality
        position = row.find('td', class_='player__position').get_text(strip=True)
        country = row.find('span', class_='player__country').get_text(strip=True)
        

        #write scraped data into file
        player_file.write(f"{player_name}, {position}, {country},")
        
        #begin second function
        scrape_premier_league_players_stats_2(player_href,player_file)


        
#second function to gain data from linked player page
def scrape_premier_league_players_stats_2(player_page,player_file):

    player_page="https:"+player_page

    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(player_page)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
 
    #finds player overview labels      
    labels = soup.find_all('div', class_='player-overview__label')

    #finds rows of player club history
    clubs_rows = soup.find_all('tr', class_='table player-club-history__table-row')
                            
    #creates empty strings for desired variables
    club_name=""
    height=""
    DoB=""
    goals=""

       
    #finds club and goals for the same season as filter rather than total goals and most recent club
    for row in clubs_rows:
        current_season = row.find('td', class_='player-club-history__season')
        if current_season and current_season.get_text(strip=True) == season[:-2] + season[:2] + season[-2:]:
            club_name = row.find('span', class_='player-club-history__team-name').get_text(strip=True)
            goals = row.find('td', class_='player-club-history__goals').get_text(strip=True)
            break
           


    #loop through all found labels to find height
    for label in labels:
        if label.get_text(strip=True).lower() == "height":
            #find the next related div that has class 'player-overview__info'
            info_div = label.find_next_sibling('div', class_='player-overview__info')
            if info_div:
                height = info_div.get_text(strip=True)
                break


    
    #loop through all found labels to find DoB
    for label in labels:
        if label.get_text(strip=True).lower() == "date of birth":
            # Find the next related div that has class 'player-overview__info'
            info_div = label.find_next_sibling('div', class_='player-overview__info')
            if info_div:
                DoB = info_div.get_text(strip=True)
                break


    #writes data into csv and then starts a new row in CSV for next player
    player_file.write(f"{club_name}, {height}, {DoB}, {goals}\n")

    #closes webpage
    driver.quit()




#open the webpage and accept cookies 

url = "https://www.premierleague.com/players"
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
accept_cookies =  driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
accept_cookies.click()
remove = driver.find_element(By.XPATH, '//*[@id="advertClose"]')
remove.click()


#list of years to collect data for
years = ['2022/23']
i=2012


for season in years:
    i=i+10

 #Open a file

    file1 = open(r"C:\Users\matts\Documents\Empirical Project\EmpiricalProject-720005462\PlayerData/PlayerData" + str(i) + ".csv", 'w')
    file1.write('Player Name, Position, Nationality, Club, Height, DoB, Goals\n')

   

#Select the year to process
    driver.execute_script("window.scrollTo(0, 0)")
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@data-dropdown-current='compSeasons']  ")))
    element.click()
    year_to_select = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.XPATH, f"//li[text()='{season}']"))
    )
    year_to_select.click()



    #Scroll to bottom and wait 60 seconds
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(60)
   
    #wait for page to refresh
    #time.sleep(10)



    #get the data
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    scrape_premier_league_players(soup,file1)
    print()

    #close the file
    file1.close()

#close web page
driver.quit()

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: unable to send message to renderer
  (Session info: chrome=124.0.6367.91)
Stacktrace:
	GetHandleVerifier [0x00007FF785151502+60802]
	(No symbol) [0x00007FF7850CAC02]
	(No symbol) [0x00007FF784F87CE4]
	(No symbol) [0x00007FF784F700D4]
	(No symbol) [0x00007FF784F7004C]
	(No symbol) [0x00007FF784F6E759]
	(No symbol) [0x00007FF784F6F3FF]
	(No symbol) [0x00007FF784F7E66E]
	(No symbol) [0x00007FF784F957CF]
	(No symbol) [0x00007FF784F9B38A]
	(No symbol) [0x00007FF784F6FB85]
	(No symbol) [0x00007FF784F953AE]
	(No symbol) [0x00007FF78501A582]
	(No symbol) [0x00007FF784FFA923]
	(No symbol) [0x00007FF784FC8FEC]
	(No symbol) [0x00007FF784FC9C21]
	GetHandleVerifier [0x00007FF78545411D+3217821]
	GetHandleVerifier [0x00007FF7854960B7+3488055]
	GetHandleVerifier [0x00007FF78548F03F+3459263]
	GetHandleVerifier [0x00007FF78520B846+823494]
	(No symbol) [0x00007FF7850D5F9F]
	(No symbol) [0x00007FF7850D0EC4]
	(No

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def scrape_premier_league_players(soup, player_file, start_index=424):
    player_rows = soup.find_all('tr', class_='player')[start_index:]
    for row in player_rows:
        player_name_link = row.find('a', class_='player__name')
        if player_name_link:
            player_name = player_name_link.get_text(strip=True)
            player_href = player_name_link.get('href')
        position = row.find('td', class_='player__position').get_text(strip=True)
        country = row.find('span', class_='player__country').get_text(strip=True)
        player_file.write(f"{player_name}, {position}, {country},")
        scrape_premier_league_players_stats_2(player_href, player_file)

def scrape_premier_league_players_stats_2(player_page, player_file):
    player_page = "https:" + player_page
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(player_page)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    labels = soup.find_all('div', class_='player-overview__label')
    clubs_rows = soup.find_all('tr', class_='table player-club-history__table-row')
    club_name = ""
    height = ""
    DoB = ""
    goals = ""
    for row in clubs_rows:
        current_season = row.find('td', class_='player-club-history__season')
        if current_season and current_season.get_text(strip=True) == season[:-2] + season[:2] + season[-2:]:
            club_name = row.find('span', class_='player-club-history__team-name').get_text(strip=True)
            goals = row.find('td', class_='player-club-history__goals').get_text(strip=True)
            break
    for label in labels:
        if label.get_text(strip=True).lower() == "height":
            info_div = label.find_next_sibling('div', class_='player-overview__info')
            if info_div:
                height = info_div.get_text(strip=True)
                break
    for label in labels:
        if label.get_text(strip=True).lower() == "date of birth":
            info_div = label.find_next_sibling('div', class_='player-overview__info')
            if info_div:
                DoB = info_div.get_text(strip=True)
                break
    player_file.write(f"{club_name}, {height}, {DoB}, {goals}\n")
    driver.quit()

url = "https://www.premierleague.com/players"
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
accept_cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')))
accept_cookies.click()
remove = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="advertClose"]')))
remove.click()

years = ['2022/23']
i = 2012

for season in years:
    i += 10
    file1 = open(r"C:\Users\matts\Documents\Empirical Project\EmpiricalProject-720005462\PlayerData/PlayerData" + str(i) + ".csv", 'a')
    driver.execute_script("window.scrollTo(0, 0)")
    element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@data-dropdown-current='compSeasons']")))
    element.click()
    year_to_select = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//li[text()='{season}']")))
    year_to_select.click()
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(60)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    scrape_premier_league_players(soup, file1, 424)
    print()
    file1.close()

driver.quit()